Sandbox for ingestion db process

**table ingestion fields.**    
index  
phash (str)  
original_filename_with_path (str)  
ingestion_date (datetime)  
discarded (bool)  
discarded_readon [duplicate | corrupt | None]  
partner_name (str)  
project_name (str)  

In [1]:
!pip install sqlalchemy pymysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 9.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.9/603.9 kB 6.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import enum
from datetime import datetime

In [3]:
from sqlalchemy import create_engine

db_user = 'user'
db_password = 'userpassword'
db_host = 'mariadb_basics'
db_port = '3306'
db_name = 'new_backend_desarrollo'

# URL de conexión para SQLAlchemy con PyMySQL
connection_url = f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

# Crear el engine de SQLAlchemy
engine = create_engine(connection_url, echo=False)

In [4]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("SELECT NOW();"))
    for row in result:
        print("Conectado correctamente. Fecha/hora del servidor:", row[0])

Conectado correctamente. Fecha/hora del servidor: 2025-05-16 17:40:31


In [8]:
from sqlalchemy import (
    Column, Integer, String, DateTime, Boolean, Enum, ForeignKey
)
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship


Base = declarative_base()

class DiscardReasonEnum(enum.Enum):
    duplicate = "duplicate"
    corrupt = "corrupt"


class Ingestion(Base):
    __tablename__ = 'ingestion'

    index = Column(Integer, primary_key=True, autoincrement=True)
    phash = Column(String(255), nullable=False)
    original_filename_with_path = Column(String(1024), nullable=False)
    ingestion_date = Column(DateTime, default=datetime.utcnow, nullable=False)
    discarded = Column(Boolean, default=False, nullable=False)
    discarded_reason = Column(Enum(DiscardReasonEnum), nullable=True)
    partner_id = Column(Integer, ForeignKey('partners.id'), nullable=False)
    project_id = Column(Integer, ForeignKey('projects.id'), nullable=False)

    partner = relationship("Partner")
    project = relationship("Project")


class Partner(Base):
    __tablename__ = 'partners'

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(100), nullable=False, unique=True)  # comentar qué opina de tener restricción de unicidad en el nombre del partner (haremos lo mismo con los proyectos?)


class Project(Base):
    __tablename__ = 'projects'

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(100), nullable=False)



In [9]:
Base.metadata.create_all(engine) #sólo crea las tablas que faltan
print("Tablas creadas correctamente.") 

Tablas creadas correctamente.


### ⬇️⬇️⬇️ Sólo si no tiene datos la db ‼️‼️‼️

In [10]:
from sqlalchemy.orm import Session

# Crear una sesión
session = Session(bind=engine)

# Crear 7 partners (nombres de personas)
partners = [
    Partner(name="Ana Torres"),
    Partner(name="Luis Gómez"),
    Partner(name="Claudia Rivera"),
    Partner(name="Jorge Martínez"),
    Partner(name="Sara Delgado"),
    Partner(name="Pedro Sánchez"),
    Partner(name="Lucía Fernández")
]

# Crear 3 proyectos
projects = [
    Project(name="mobile"),
    Project(name="crop"),
    Project(name="slit")
]

# Agregar a la sesión
session.add_all(partners + projects)

# Confirmar los cambios en la base de datos
try:
    session.commit()
    print("Partners y proyectos insertados correctamente.")
except Exception as e:
    session.rollback()
    print("Error al insertar datos:", e)
finally:
    session.close()

Partners y proyectos insertados correctamente.


### Insertemos ahora fotos 

In [11]:
import random
from sqlalchemy.orm import Session

def insertar_fotos(session, lista_de_fotos):
    """
    Inserta registros en la tabla Ingestion.

    :param session: Objeto de sesión SQLAlchemy.
    :param lista_de_fotos: Lista de dicts con claves:
        - phash (str)
        - original_filename_with_path (str)
        - discarded (bool, opcional)
        - discarded_reason (str: 'duplicate' | 'corrupt', opcional)
    """

    # Obtener todos los partners y proyectos disponibles
    partners = session.query(Partner).all()
    projects = session.query(Project).all()

    if not partners or not projects:
        print("ERROR: Debes tener al menos un partner y un proyecto registrados.")
        return

    for foto in lista_de_fotos:
        # Validación básica
        if not all(k in foto for k in ["phash", "original_filename_with_path"]):
            print("Registro incompleto:", foto)
            continue

        # Convertir razón de descarte si está presente
        reason = foto.get("discarded_reason")
        if reason:
            try:
                reason_enum = DiscardReasonEnum(reason)
            except ValueError:
                print(f"Razón inválida: {reason}")
                reason_enum = None
        else:
            reason_enum = None

        # Crear objeto Ingestion
        nuevo_registro = Ingestion(
            phash=foto["phash"],
            original_filename_with_path=foto["original_filename_with_path"],
            discarded=foto.get("discarded", False),
            discarded_reason=reason_enum,
            partner=random.choice(partners),
            project=random.choice(projects)
        )

        session.add(nuevo_registro)

    try:
        session.commit()
        print(f"{len(lista_de_fotos)} registros insertados correctamente.")
    except Exception as e:
        session.rollback()
        print("Error al insertar:", e)

In [12]:
# Crear sesión
session = Session(bind=engine)

# Datos simulados
fotos = [
    {
        "phash": "a1b2c3d4",
        "original_filename_with_path": "/data/images/foto1.jpg"
    },
    {
        "phash": "e5f6g7h8",
        "original_filename_with_path": "/data/images/foto2.jpg",
        "discarded": True,
        "discarded_reason": "duplicate"
    },
    {
        "phash": "i9j0k1l2",
        "original_filename_with_path": "/data/images/foto3.jpg",
        "discarded": True,
        "discarded_reason": "corrupt"
    }
]

# Llamar a la función
insertar_fotos(session, fotos)

# Cerrar sesión
session.close()

3 registros insertados correctamente.


In [13]:
fotos_2 = [
    {
        "phash": "001a9fefb4c1d2e3",
        "original_filename_with_path": "/data/images/exp_001.jpg"
    },
    {
        "phash": "002b8eee91c2a3b4",
        "original_filename_with_path": "/data/images/exp_002.jpg"
    },
    {
        "phash": "003c7ddd82d3b4c5",
        "original_filename_with_path": "/data/images/exp_003.jpg",
        "discarded": True,
        "discarded_reason": "duplicate"
    },
    {
        "phash": "004d6ccc73e4c5d6",
        "original_filename_with_path": "/data/images/exp_004.jpg"
    },
    {
        "phash": "005e5bbb64f5d6e7",
        "original_filename_with_path": "/data/images/exp_005.jpg",
        "discarded": True,
        "discarded_reason": "corrupt"
    },
    {
        "phash": "006f4aaa55a6e7f8",
        "original_filename_with_path": "/data/images/exp_006.jpg"
    },
    {
        "phash": "007a3b9934b7f801",
        "original_filename_with_path": "/data/images/exp_007.jpg"
    },
    {
        "phash": "008b2c8823c80112",
        "original_filename_with_path": "/data/images/exp_008.jpg",
        "discarded": True,
        "discarded_reason": "duplicate"
    },
    {
        "phash": "009c1d7712d91223",
        "original_filename_with_path": "/data/images/exp_009.jpg"
    },
    {
        "phash": "010d0e6601ea1234",
        "original_filename_with_path": "/data/images/exp_010.jpg"
    }
]

In [14]:
session = Session(bind=engine)
insertar_fotos(session, fotos_2)
session.close()

10 registros insertados correctamente.


### Métricas  
#### Total de fotos por partner

In [15]:
from sqlalchemy import func

session = Session(bind=engine)

resultados = (
    session.query(
        Partner.name,
        func.count(Ingestion.index).label("total_fotos")
    )
    .join(Ingestion)
    .group_by(Partner.id)
    .order_by(func.count(Ingestion.index).desc())
    .all()
)

for nombre, total in resultados:
    print(f"{nombre}: {total} fotos")

Luis Gómez: 4 fotos
Jorge Martínez: 4 fotos
Lucía Fernández: 2 fotos
Pedro Sánchez: 2 fotos
Claudia Rivera: 1 fotos


#### Total de fotos descartadas por partner

In [16]:
resultados = (
    session.query(
        Partner.name,
        func.count(Ingestion.index).label("fotos_descartadas")
    )
    .join(Ingestion)
    .filter(Ingestion.discarded == True)
    .group_by(Partner.id)
    .order_by(func.count(Ingestion.index).desc())
    .all()
)

for nombre, total in resultados:
    print(f"{nombre}: {total} descartadas")

Jorge Martínez: 2 descartadas
Lucía Fernández: 1 descartadas
Luis Gómez: 1 descartadas
Pedro Sánchez: 1 descartadas


#### Fotos aportadas por partner y proyecto

In [17]:
resultados = (
    session.query(
        Partner.name,
        Project.name,
        func.count(Ingestion.index).label("total_fotos")
    )
    .join(Ingestion, Ingestion.partner_id == Partner.id)
    .join(Project, Ingestion.project_id == Project.id)
    .group_by(Partner.name, Project.name)
    .order_by(Partner.name, Project.name)
    .all()
)

for partner, proyecto, total in resultados:
    print(f"{partner} - {proyecto}: {total} fotos")

Claudia Rivera - slit: 1 fotos
Jorge Martínez - mobile: 1 fotos
Jorge Martínez - slit: 3 fotos
Lucía Fernández - mobile: 1 fotos
Lucía Fernández - slit: 1 fotos
Luis Gómez - crop: 2 fotos
Luis Gómez - mobile: 1 fotos
Luis Gómez - slit: 1 fotos
Pedro Sánchez - slit: 2 fotos


#### Fotos por mes

In [18]:
from sqlalchemy import extract

resultados = (
    session.query(
        extract('year', Ingestion.ingestion_date).label('año'),
        extract('month', Ingestion.ingestion_date).label('mes'),
        func.count(Ingestion.index).label('total_fotos')
    )
    .group_by('año', 'mes')
    .order_by('año', 'mes')
    .all()
)

for año, mes, total in resultados:
    print(f"{int(año)}-{int(mes):02d}: {total} fotos")

2025-05: 13 fotos


#### Fotos por mes y partner

In [19]:
resultados = (
    session.query(
        Partner.name,
        extract('year', Ingestion.ingestion_date).label('año'),
        extract('month', Ingestion.ingestion_date).label('mes'),
        func.count(Ingestion.index).label('total_fotos')
    )
    .join(Ingestion)
    .group_by(Partner.name, 'año', 'mes')
    .order_by(Partner.name, 'año', 'mes')
    .all()
)

for partner, año, mes, total in resultados:
    print(f"{partner} - {int(año)}-{int(mes):02d}: {total} fotos")

Claudia Rivera - 2025-05: 1 fotos
Jorge Martínez - 2025-05: 4 fotos
Lucía Fernández - 2025-05: 2 fotos
Luis Gómez - 2025-05: 4 fotos
Pedro Sánchez - 2025-05: 2 fotos


#### Fotos descartadas por mes y razón

In [20]:
resultados = (
    session.query(
        extract('year', Ingestion.ingestion_date).label('año'),
        extract('month', Ingestion.ingestion_date).label('mes'),
        Ingestion.discarded_reason,
        func.count(Ingestion.index).label('descartadas')
    )
    .filter(Ingestion.discarded == True)
    .group_by('año', 'mes', Ingestion.discarded_reason)
    .order_by('año', 'mes', Ingestion.discarded_reason)
    .all()
)

for año, mes, razon, total in resultados:
    razon_str = razon.value if razon else "sin razón"
    print(f"{int(año)}-{int(mes):02d} | {razon_str}: {total}")

2025-05 | duplicate: 3
2025-05 | corrupt: 2
